In [25]:
# IMPORT LIBRARIES
import pandas as pd
from pathlib import Path
import os
import sqlite3
import numpy as np
from pandas.api.types import is_numeric_dtype


In [26]:
# CREATE SQL FIELDS FROM CSV

def create_sql_fields(df):                                          # gather the headers of the CSV and create two strings 
    fields_sql = []                                                 # str1 = var1 TYPE, va2, TYPE ...
    header_names = []                                               # str2 = var1, var2, var3, var4
    for col in range(0,len(df.columns)):
        fields_sql.append(df.columns[col])
        fields_sql.append(str(df.dtypes[col]))

        header_names.append(df.columns[col])
        if col != len(df.columns)-1:
            fields_sql.append(',')
            header_names.append(',')

    fields_sql = ' '.join(fields_sql)
    fields_sql = fields_sql.replace('int64','integer')
    fields_sql = fields_sql.replace('float64','integer')
    fields_sql = fields_sql.replace('object','text')

    header_sql_string = '(' + ''.join(header_names) + ')'
    
    return fields_sql, header_sql_string


In [27]:
# LOAD CSV 
csv_filedir = os.path.join('C:\\Code\\Git\\DataScience_code_academy\\Data_Science_skills\\NY_Restaurants\\nomnom.csv')
data = pd.read_csv(csv_filedir)   
csv_df = pd.DataFrame(data)


In [ ]:
# STRUTURE DATA
for col in csv_df.columns:
    if is_numeric_dtype(csv_df[col].dtype)==0:
        csv_df[col] = csv_df[col].str.replace("\'", "\'\'")     # replace " ' " for " '' " for SQL in each column
        csv_df[col] = csv_df[col].str.replace("\\", "\\\'")     # replace " ' " for " '' " for SQL in each column

csv_df = csv_df.fillna('NULL')                              # make NaN = to 'NULL' for SQL format 



In [29]:
# CREATE / OPEN DATABASE
[path,filename] = os.path.split(csv_filedir)                # define path and filename for .db file
[filename,_] = os.path.splitext(filename)
database_filedir = os.path.join(path, filename + '.db')

conn = sqlite3.connect(database_filedir)                    # connect to SQL server

[fields_sql, header_sql_string] = create_sql_fields(csv_df)

create_sql = ''.join(['CREATE TABLE IF NOT EXISTS ' + filename + ' (' + fields_sql + ')'])
cursor = conn.cursor()
_ = cursor.execute(create_sql)

   

In [ ]:
# INSERT EACH ROW IN THE SQL DATABASE
csv_tuple = csv_df.values.tolist()
for irow in csv_tuple:
    insert_values_string = ''.join(['INSERT INTO ', filename, header_sql_string, ' VALUES'])
    insert_sql = f"{insert_values_string} {tuple(irow)}"
    print(insert_sql)
    cursor.execute(insert_sql)

# COMMIT CHANGES TO DATABASE AND CLOSE CONNECTION
conn.commit()
conn.close()

print('\n' + csv_filedir + ' \n converted to \n' + database_filedir)

In [ ]:
# random data tyoe conversions
insert_sql = f"{insert_values_string} {irow[1]}, '{irow[2]}','{irow[3]}', {irow[4]}, '{irow[5]}' )"
print('insert_sql' , insert_sql)
print('type insert_sql = ' + str(type(insert_sql)))
print('   ')

print('{tuple(irow)} = ', {tuple(irow)})
print('type tuple(irow) = ', type({tuple(irow)}))
print('   ')

print('type irow[3] = ' + str(type(irow[3])))
print('   ')

row = ({irow[1]}, {irow[2]},{irow[3]}, {irow[4]}, {irow[5]})
print('row = ' + str(row))
print('type row = ' + str(type(row)))

row_set = set(row)
print('row_set = ' + str(row_set))
print('type row_set = ' + str(type(row_set)))

'Japanese'